<a href="https://colab.research.google.com/github/rahulpandey89/Age_and_Gender_Prediction/blob/master/table_detection_using_Yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision
!pip install datasets  # Hugging Face dataset ke liye
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.3/949.3 kB 24.6 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

# Dataset download karo
dataset = load_dataset("ucsahin/pubtables-detection-1500-samples")
print(dataset)  # Structure check karo

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/225M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'objects'],
        num_rows: 1500
    })
})


In [4]:
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from datasets import load_dataset

# Dataset load karo
#dataset = load_dataset("uwgwt/PubTables-Detection-1500-Samples")
train_data = dataset["train"]

# Train data ko list mein convert karo
train_data_list = [train_data[i] for i in range(len(train_data))]

# Train aur validation mein split karo (80% train, 20% val)
train_examples, val_examples = train_test_split(train_data_list, test_size=0.2, random_state=42)
print(f"Train samples: {len(train_examples)}, Validation samples: {len(val_examples)}")

# Directories banao
os.makedirs("/content/yolo_dataset/images/train", exist_ok=True)
os.makedirs("/content/yolo_dataset/labels/train", exist_ok=True)
os.makedirs("/content/yolo_dataset/images/val", exist_ok=True)
os.makedirs("/content/yolo_dataset/labels/val", exist_ok=True)

def convert_to_yolo(example, split="train", idx=0):
    img = example["image"]
    img_width, img_height = img.size
    img_path = f"/content/yolo_dataset/images/{split}/{idx}.jpg"
    label_path = f"/content/yolo_dataset/labels/{split}/{idx}.txt"

    img.save(img_path)

    objects = example["objects"]
    bboxes = objects["bbox"]

    with open(label_path, "w") as f:
        for bbox in bboxes:
            x_min, y_min, x_max, y_max = bbox
            width = x_max - x_min
            height = y_max - y_min
            x_center = (x_min + width / 2) / img_width
            y_center = (y_min + height / 2) / img_height
            width = width / img_width
            height = height / img_height
            f.write(f"0 {x_center} {y_center} {width} {height}\n")

# Train aur val data process karo
for idx, example in enumerate(train_examples):
    convert_to_yolo(example, "train", idx)
for idx, example in enumerate(val_examples):
    convert_to_yolo(example, "val", idx)

# YOLO configuration file banao
with open("/content/yolo_dataset/data.yaml", "w") as f:
    f.write("""
train: /content/yolo_dataset/images/train
val: /content/yolo_dataset/images/val
nc: 1
names: ["table"]
""")

Train samples: 1200, Validation samples: 300


In [8]:
print(dataset["train"][0])



{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=771x1000 at 0x7EE1B590C310>, 'objects': {'bbox': [[75.53573789752897, 805.4076686049953, 580.396252241291, 882.670855281329]], 'categories': 'table'}}


In [5]:
!pip install ultralytics

from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(data="/content/yolo_dataset/data.yaml", epochs=50, imgsz=640, batch=16, device=0)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 108MB/s]


Ultralytics 8.3.92 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/yolo_dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

100%|██████████| 755k/755k [00:00<00:00, 20.8MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        300        357      0.996          1      0.995      0.994



50 epochs completed in 0.112 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.92 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:01<00:00,  6.71it/s]


                   all        300        357      0.994          1      0.995      0.995
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bca7878ecd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [6]:
from ultralytics import YOLO
model = YOLO("/content/runs/detect/train/weights/best.pt")
results = model.predict("/content/yolo_dataset/images/val", save=True, device=0)


image 1/300 /content/yolo_dataset/images/val/0.jpg: 640x512 2 tables, 46.0ms
image 2/300 /content/yolo_dataset/images/val/1.jpg: 640x512 1 table, 7.6ms
image 3/300 /content/yolo_dataset/images/val/10.jpg: 640x512 1 table, 7.3ms
image 4/300 /content/yolo_dataset/images/val/100.jpg: 640x512 2 tables, 7.3ms
image 5/300 /content/yolo_dataset/images/val/101.jpg: 640x512 1 table, 7.3ms
image 6/300 /content/yolo_dataset/images/val/102.jpg: 640x512 1 table, 7.3ms
image 7/300 /content/yolo_dataset/images/val/103.jpg: 640x512 2 tables, 7.7ms
image 8/300 /content/yolo_dataset/images/val/104.jpg: 640x512 1 table, 7.2ms
image 9/300 /content/yolo_dataset/images/val/105.jpg: 640x512 1 table, 7.2ms
image 10/300 /content/yolo_dataset/images/val/106.jpg: 640x512 2 tables, 7.2ms
image 11/300 /content/yolo_dataset/images/val/107.jpg: 640x512 1 table, 7.2ms
image 12/300 /content/yolo_dataset/images/val/108.jpg: 640x512 2 tables, 7.4ms
image 13/300 /content/yolo_dataset/images/val/109.jpg: 640x512 2 tables

In [7]:
#!pip install pdf2image

#from pdf2image import convert_from_path
#images = convert_from_path("/content/yolo_dataset/images/val/0.jpg", dpi=300)
#images[0].save("/content/yolo_dataset/images/val/0.jpg", "JPEG")

results = model.predict("/content/yolo_dataset/images/val/0.jpg", save=True, device=0)


image 1/1 /content/yolo_dataset/images/val/0.jpg: 640x512 2 tables, 8.6ms
Speed: 3.1ms preprocess, 8.6ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 512)
Results saved to runs/detect/predict


In [14]:
image=Image.open('/content/yolo_dataset/images/val/0.jpg')

In [15]:
from PIL import Image

In [16]:
#results = model.predict("test_image.jpg", save=True, device=0)

# Bounding box coordinates nikalo
boxes = results[0].boxes.xyxy.cpu().numpy()  # [x_min, y_min, x_max, y_max]
if len(boxes) > 0:
    x_min, y_min, x_max, y_max = boxes[0]  # Pehla table ka box
    table_img = image.crop((x_min, y_min, x_max, y_max))
    table_img.save("cropped_table.jpg")
else:
    print("Koi table detect nahi hua!")

In [20]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [21]:
!apt-get install -y tesseract-ocr
!pip install pytesseract ultralytics opencv-python pandas
import cv2
import pandas as pd


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,660 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 125044 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [24]:
import pytesseract
table_cv = cv2.imread("/content/cropped_table.jpg")
text = pytesseract.image_to_string(table_cv)
print("Extracted Text:\n", text)

# Text ko rows aur columns mein parse karo
lines = text.strip().split("\n")
table_data = [line.split() for line in lines if line.strip()]

if table_data and len(table_data) > 1:
        # Max columns ka size nikalo
    max_cols = max(len(row) for row in table_data)
    # Header ko dynamically set karo
    headers = [f"Col_{i+1}" for i in range(max_cols)] if len(table_data[0]) < max_cols else table_data[0]
    # Data rows ko pad karo agar columns kam hain
    data_rows = [row + [""] * (max_cols - len(row)) if len(row) < max_cols else row[:max_cols] for row in table_data[1:]]

    df = pd.DataFrame(data_rows, columns=headers[:max_cols])
    df.to_csv("output_table.csv", index=False)
    print("Table saved: output_table.csv")
else:
    print("OCR se table data extract nahi hua!")


Extracted Text:
  

 

us Tota
Chest CT Large Moderate Mid (en
Large 2 ° ° 2
Moderate 1 5 1 7
Mid ° 2 10 2
Total (US). ‘~ ? ay a

Table saved: output_table.csv
